In [ ]:
%run ./config/config.ipynb

In [ ]:
# using SQLContext to read parquet file
sqlContext = SQLContext(sc)

In [ ]:
%run ./variables/path_variable.ipynb

In [ ]:
%run ./variables/TempTable.ipynb

In [ ]:
# to read parquet file
df_raw = spark.read.parquet(path_raw)

In [ ]:
df_raw.count()

In [ ]:
df_raw.registerTempTable(TempTableRaw)

In [ ]:
df_raw_sql = spark.sql(" SELECT * FROM raw_delta ")

In [ ]:
df_raw_sql.count()

In [ ]:
%run ./variables/today.ipynb

In [ ]:
path_exist = DeltaTable.isDeltaTable(spark, path_refined)

In [ ]:
if path_exist == False:
    df_raw_select = df_raw_sql.select(df_raw_sql['*'])
else:
    df_raw_select = df_raw_sql.select(df_raw_sql['*']).filter(df_raw_sql['last_access'] == today)

In [ ]:
df_raw_select.count()

In [ ]:
# Grava os arquivos na camada Raw
df_raw_select.write \
             .format("delta") \
             .mode("append") \
             .partitionBy('last_access') \
             .save(path_refined)

In [ ]:
sc.stop()